In [1]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 --force-reinstall --upgrade --no-cache-dir --verbose

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.1 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 3.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 6.1 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 15.7 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 25.8 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 5.8 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-wcfv29su/overlay/local/bin
    changing mode of /tmp/pip-build-env-wcfv29su/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-wcfv29su/overlay/lo

In [2]:
# For download the models
!pip install huggingface_hub

Выбирем лучшую модель из тех, что можем себе позволить



![asd](https://substackcdn.com/image/fetch/f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2Fc24dac6d-6b5e-4b5f-938c-05951c938a9e_1085x543.png)






# Квантизация

На huggingface большой список кванторизированных моделей, я возьму [llama-2-7b-chat-GGML](https://huggingface.co/TheBloke/Llama-2-7B-chat-GGML)

In [3]:
model_name_or_path = "TheBloke/Llama-2-7B-chat-GGML"
model_basename = "llama-2-7b-chat.ggmlv3.q5_1.bin" # the model is in bin format

Скачиваем

In [4]:
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

llama-2-7b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/5.06G [00:00<?, ?B/s]

Инициализируем модель

In [5]:
# GPU
from llama_cpp import Llama
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2,
    n_batch=512,
    n_gpu_layers=43,
    n_ctx=4096, # Максимальное количество токенов на входе
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


Скачаем датасет

In [8]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.4 MB/s eta 0:00:00


In [9]:
from datasets import load_dataset

dataset = load_dataset("SetFit/amazon_reviews_multi_en", split='train')

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Возьмем пример отзыва

In [15]:
sample_review = dataset[2]['text']
sample_review

"I received my first order of this product and it was broke so I ordered it again. The second one was broke in more places than the first. I can't blame the shipping process as it's shrink wrapped and boxed."

Для задачи выделения плюсов и минусов продукта я подобрал, методом проб и ошибок, следующий промпт

In [16]:
example_input = '''
I've bought this headset yesterday. The basses are very good, and they look very pretty. May be it would be better to add headphone case as well.
'''

example_answer = '''
Pros:
1) Basses are very good
2) Pretty design
'''

In [17]:
prompt = '''
Provide a list of 0-6 pros for the given product based on its review. Each point should be concise, consisting of 2-3 words.

'''


review = f'''
{sample_review}
'''

prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: {prompt}
 {example_input}

ASSISTANT: {example_answer}

USER: {prompt}
 {review}

ASSISTANT:
'''

Генерируем ответ

In [18]:
response = lcpp_llm(
    prompt=prompt_template,
    max_tokens=512,
    temperature=0.5,
    top_p=0.95,
    repeat_penalty=1.2,
    top_k=50,
    stop = ['USER:'], # Останавливаемся, если увидели специальный токен "USER:"
    echo=True # возвращаем ответ вместе с промптом
)

ans = response["choices"][0]["text"].split('ASSISTANT:')[-1]
print(ans)


Pros:
1) Fast delivery
2) Good packaging


In [19]:
example_input = '''
I've bought this headset yesterday. The basses are very good, and they look very pretty. May be it would be better to add headphone case as well.
'''

example_answer = '''
Cons:
1) No case included
'''

In [20]:
prompt = '''
Provide a list of 0-6 cons for the given product based on its review. Each point should be concise, consisting of 2-3 words.

'''


review = f'''
{sample_review}
'''

prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: {prompt}
 {example_input}

ASSISTANT: {example_answer}

USER: {prompt}
 {review}

ASSISTANT:
'''

In [21]:
response = lcpp_llm(
    prompt=prompt_template,
    max_tokens=512,
    temperature=0.5,
    top_p=0.95,
    repeat_penalty=1.2,
    top_k=50,
    stop = ['USER:'], # Останавливаемся, если увидели специальный токен "USER:"
    echo=True # возвращаем ответ вместе с промптом
)

ans1 = response["choices"][0]["text"].split('ASSISTANT:')[-1]
print(ans1)

Llama.generate: prefix-match hit



Cons:
1) Poor quality control
2) Multiple breakages


# Итоговый результат

In [22]:
print(ans + '\n' + ans1)


Pros:
1) Fast delivery
2) Good packaging

Cons:
1) Poor quality control
2) Multiple breakages


# Выводы.
Правильный промпт может заставить модель выдавать нам то что нужно. На первых порах, модель выдавала плюсы и минусы не в виде списка, а просто перечислением. Создание видимости того, что модель уже общалась с нами, очень помогло.